In [10]:
%load_ext autoreload
%autoreload 2
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-864c07c4-8eeb-5b23-8d57-eaeb942a9a0f"
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import yaml
from hydra.utils import instantiate
from PIL import Image
from torch.utils.data import DataLoader, Dataset

from br.features.archetype import AA_Fast
from br.features.plot import collect_outputs, plot, plot_stratified_pc
from br.features.reconstruction import stratified_latent_walk
from br.features.utils import (
    normalize_intensities_and_get_colormap,
    normalize_intensities_and_get_colormap_apply,
)
from br.models.compute_features import compute_features, get_embeddings
from br.models.load_models import get_data_and_models
from br.models.save_embeddings import (
    get_pc_loss,
    get_pc_loss_chamfer,
    save_embeddings,
    save_emissions,
)
from br.models.utils import get_all_configs_per_dataset

device = "cuda:0"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data and models

In [11]:
os.chdir("/allen/aics/modeling/ritvik/projects/benchmarking_representations/")
save_path = "./test_polymorphic_save_embeddings/"

In [19]:
dataset_name = "other_polymorphic"
batch_size = 2
debug = False
results_path = "/allen/aics/modeling/ritvik/projects/benchmarking_representations/configs/results/"
data_list, all_models, run_names, model_sizes = get_data_and_models(
    dataset_name, batch_size, results_path, debug
)

In [20]:
data_list = data_list[1:]
all_models = all_models[1:]
run_names = run_names[1:]

# Compute embeddings and emissions

In [21]:
from br.models.save_embeddings import save_embeddings

splits_list = ["test"]
splits_list = ["train","val", "test"]
meta_key = None
eval_scaled_img = [False] * 5

gt_mesh_dir = "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/var_blobby_noalign/meshes"
gt_sampled_pts_dir = "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/sampled_pcs/blobby_var_noalign/1_res/0"
gt_scale_factor_dict_path = "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/blobby_var_scale_factor_32res_noalign.npz"

eval_scaled_img_params = [
    {
        "eval_scaled_img_model_type": "iae",
        "eval_scaled_img_resolution": 32,
        "gt_mesh_dir": gt_mesh_dir,
        "gt_scale_factor_dict_path": None,
        "gt_sampled_pts_dir": gt_sampled_pts_dir,
        "mesh_ext": "stl",
    },
    {
        "eval_scaled_img_model_type": "sdf",
        "eval_scaled_img_resolution": 32,
        "gt_mesh_dir": gt_mesh_dir,
        "gt_scale_factor_dict_path": gt_scale_factor_dict_path,
        "gt_sampled_pts_dir": None,
        "mesh_ext": "stl",
    },
    {
        "eval_scaled_img_model_type": "seg",
        "eval_scaled_img_resolution": 32,
        "gt_mesh_dir": gt_mesh_dir,
        "gt_scale_factor_dict_path": gt_scale_factor_dict_path,
        "gt_sampled_pts_dir": None,
        "mesh_ext": "stl",
    },
    {
        "eval_scaled_img_model_type": "sdf",
        "eval_scaled_img_resolution": 32,
        "gt_mesh_dir": gt_mesh_dir,
        "gt_scale_factor_dict_path": gt_scale_factor_dict_path,
        "gt_sampled_pts_dir": None,
        "mesh_ext": "stl",
    },
    {
        "eval_scaled_img_model_type": "seg",
        "eval_scaled_img_resolution": 32,
        "gt_mesh_dir": gt_mesh_dir,
        "gt_scale_factor_dict_path": gt_scale_factor_dict_path,
        "gt_sampled_pts_dir": None,
        "mesh_ext": "stl",
    },
]
loss_eval_list = [torch.nn.MSELoss(reduction="none")] * 5
sample_points_list = [False] * 5
skew_scale = None
save_embeddings(
    save_path,
    data_list,
    all_models,
    run_names,
    debug,
    splits_list,
    device,
    meta_key,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

Processing train


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 13306/13306 [22:03<00:00, 10.05it/s]


Processing val


  3%|███▌                                                                                                          | 91/2854 [00:09<04:34, 10.07it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f51edf59d80>
Traceback (most recent call last):
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/process.py", line 160, in is_alive

    if w.is_alive():
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f51edf59d80>
Traceback (most recent call last):
  File "/home/ritvik.vasan/anaco

Processing test


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 13306/13306 [06:57<00:00, 31.89it/s]


Processing val


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2854/2854 [01:28<00:00, 32.39it/s]


Processing test


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2853/2853 [01:27<00:00, 32.59it/s]


In [14]:
run_names

['equiv_vn',
 'CNN_SO3_pad35_sdf',
 'CNN_SO3_pad35_seg',
 'CNN_sdf_pad35',
 'CNN_seg_pad35']

In [22]:
run_names

['CNN_SO3_pad35_sdf', 'CNN_SO3_pad35_seg', 'CNN_sdf_pad35', 'CNN_seg_pad35']

# Latent walks

In [52]:
# Load model and embeddings

run_names = ["equiv_vn"]
DATASET_INFO = get_all_configs_per_dataset(results_path)
all_ret, df = get_embeddings(run_names, dataset_name, DATASET_INFO, save_path)

In [53]:
all_ret['structure_name'].value_counts()

NPM1       11814
LAMP1      10114
FBL         9923
ST6GAL1     6175
Name: structure_name, dtype: int64

In [54]:
[i for i in all_ret.columns if "path" in i]

['fov_path',
 'fov_seg_path',
 'mesh_path_noalign',
 'meta_plate_path_mask',
 'meta_plate_path_segmentation',
 'points_sdf_noalign_path',
 'processed_sdf_path_noalign_48res_51pad',
 'processed_sdf_path_noalign_64res_67pad',
 'processed_seg_path_noalign_48res_51pad',
 'processed_seg_path_noalign_64res_67pad',
 'sdf_path_noalign_32res',
 'sdf_path_noalign_48res',
 'sdf_path_noalign_64res',
 'seg_path_noalign_32res',
 'seg_path_noalign_48res',
 'seg_path_noalign_64res',
 'struct_seg_path']

In [55]:
from sklearn.decomposition import PCA
from tqdm import tqdm
from cyto_dl.image.transforms import RotationMask
from skimage.io import imread
from br.data.utils import mesh_seg_model_output
import pyvista as pv

this_save_path = Path(save_path) / Path("latent_walks")
this_save_path.mkdir(parents=True, exist_ok=True)


for num_pieces in all_ret['structure_name'].unique():
    this_sub_m = all_ret.loc[all_ret['structure_name'] == num_pieces].reset_index(drop=True)
    all_features = this_sub_m[[i for i in this_sub_m.columns if "mu" in i]].values
    latent_dim=512
    dim_size = latent_dim
    x_label = 'pcloud'
    pca = PCA(n_components=dim_size)
    pca_features = pca.fit_transform(all_features)
    pca_std_list = pca_features.std(axis=0)
    for rank in [0, 1]:
        all_xhat = []
        all_closest_real = []
        all_closest_img = []
        latent_walk_range = [-2, 0, 2]
        for value_index, value in enumerate(
                tqdm(latent_walk_range, total=len(latent_walk_range))
            ):
            z_inf = torch.zeros(1, dim_size)
            z_inf[:, rank] += value * pca_std_list[rank]
            z_inf = pca.inverse_transform(z_inf).numpy()
        
            dist = (all_features - z_inf) ** 2
            dist = np.sum(dist, axis=1)
            closest_idx = np.argmin(dist)
            closest_real_id = this_sub_m.iloc[closest_idx]['CellId']
            mesh = pv.read(all_ret.loc[all_ret['CellId'] == closest_real_id]['mesh_path_noalign'].iloc[0])
            mesh.save(this_save_path / Path(f'{num_pieces}_{rank}_{value_index}.ply'))
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 13.81it/s]


In [70]:
# num_pieces = 4.0
struct = "LAMP1"
rank = 0
bin_ = 2
this_mesh_path = this_save_path / Path(f'{struct}_{rank}_{bin_}.ply')
this_mesh_path = './' + str(this_mesh_path)

mitsuba_save_path = this_save_path / Path('mitsuba')
mitsuba_save_path.mkdir(parents=True, exist_ok=True)
mitsuba_save_path = './' + str(mitsuba_save_path)
name = f"{struct}_{rank}_{bin_}"


plot(str(this_mesh_path), mitsuba_save_path, 65, 0, None, name)

# Archetype

In [31]:
from br.features.archetype import AA_Fast
n_archetypes = 4
matrix = all_ret[[i for i in all_ret.columns if "mu" in i]].values
aa = AA_Fast(n_archetypes, max_iter=1000, tol=1e-6).fit(matrix)

import pandas as pd
archetypes_df = pd.DataFrame(aa.Z, columns = [f'mu_{i}' for i in range(matrix.shape[1])])

In [40]:
archetypes_df

,mu_0,mu_1,mu_2,mu_3,mu_4,mu_5,mu_6,mu_7,mu_8,mu_9,...,mu_502,mu_503,mu_504,mu_505,mu_506,mu_507,mu_508,mu_509,mu_510,mu_511
0,0.016720,0.013911,0.015173,0.016448,0.013668,0.013938,0.014895,0.013026,0.019262,0.024216,...,0.016125,0.025148,0.014622,0.017662,0.021135,0.019173,0.015466,0.018578,0.017736,0.026095
1,0.011478,0.021628,0.008261,0.010213,0.012635,0.011072,0.014946,0.010943,0.014555,0.011230,...,0.009062,0.020621,0.011007,0.012977,0.014252,0.009047,0.010252,0.011973,0.014670,0.013925
2,0.014800,0.029973,0.018342,0.015434,0.012984,0.021673,0.016685,0.017274,0.014409,0.019791,...,0.023301,0.022890,0.019476,0.015652,0.017187,0.018769,0.016542,0.017992,0.020920,0.018426
3,0.008071,0.021726,0.016243,0.018224,0.021039,0.020376,0.022450,0.024019,0.013192,0.016768,...,0.012576,0.033081,0.018664,0.015084,0.020398,0.016422,0.015154,0.020113,0.017388,0.017130


In [32]:
this_save_path = Path(save_path) / Path("archetypes")
this_save_path.mkdir(parents=True, exist_ok=True)

all_features =  matrix
for i in range(n_archetypes):
    this_mu = archetypes_df.iloc[i].values
    dist = (all_features - this_mu) ** 2
    dist = np.sum(dist, axis=1)
    closest_idx = np.argmin(dist)
    closest_real_id = all_ret.iloc[closest_idx]['CellId']
    mesh = pv.read(all_ret.loc[all_ret['CellId'] == closest_real_id]['mesh_path_noalign'].iloc[0])
    mesh.save(this_save_path / Path(f'{i}.ply'))
        

In [44]:
this_save_path

PosixPath('test_polymorphic_save_embeddings/archetypes')

In [51]:
from br.visualization.mitsuba_render_image import plot
# num_pieces = 4.0
arch = "3"
this_mesh_path = this_save_path / Path(f'{arch}.ply')
this_mesh_path = './' + str(this_mesh_path)

mitsuba_save_path = this_save_path / Path('mitsuba')
mitsuba_save_path.mkdir(parents=True, exist_ok=True)
mitsuba_save_path = './' + str(mitsuba_save_path)
name = f"{arch}"


plot(str(this_mesh_path), mitsuba_save_path, 10, 0, None, name)